In [51]:
# make-dense(csr)
import re
from codegen.iegenlib import IEGenLib
from tools.system import run

#icsr = 'Icsr := {[i,j]: 0 <= i < N_R && index(i) <= j < index(i+1) && j >= 0 && j < NNZ}'
# icsr = 'Icsr := {[i,j]: 0 <= i < N_R && index(i) <= j < index(i+1)}'
# tdense = 'Tdense := {[i,j] -> [i,k,j]: 0 <= k < N_C && k = col(j)}'
# idense = 'Idense := {[i,k,j]: k - col(j) = 0 && i >= 0 && k >= 0 && j - index(i) >= 0 && -i + N_R - 1 >= 0 && -k + N_C - 1 >= 0 && -j + index(i + 1) - 1 >= 0 };'
# ttile = 'Ttile := {[i,k,j] -> [ii,kk,i,k,j]: 0 <= ii < floor(N_R/8) && 0 <= kk < floor(N_C/8) && 8 * ii <= i < min(N_R - 1, 8 * ii + 7) && 8 * kk <= k < min(N_C - 1, 8 * kk + 7)}'

# # Perform the make-dense transformation with IEGenLib/Omega:
# iegen = IEGenLib()
# iegen.add(icsr)
# iegen.add(tdense)
# (resid, result) = iegen.apply('Tdense', 'Icsr', 'Idense')
# assign = '%s := %s;' % (resid, result)
# print("(IEGenLib) " + assign)
#omcode = iegen.to_omega(resid)

#Idense := { [i, k, j] : k - col(i,k,j) = 0 && i >= 0 && k >= 0 && j - index(i) >= 0 && -i + N_R - 1 >= 0 && -k + N_C - 1 >= 0 && -j + index1(i) - 1 >= 0 && -index(i) + index1(i) - 1 >= 0 };
#codegen(Idense) given {[i,k,j]: index1(i) >= index(i)+1 && N_C >= 1};
#Itile := { [ii,kk,i,k,j]: exists(ri,ck: 0 <= ri < 8 && i = ii*8+ri && 0 <= ck < 8 && k = kk*8+ck) && k - col(ii,kk,i,k,j) = 0 && i >= 0 && k >= 0 && index(ii,kk,i) >= 0 && j - index(ii,kk,i) >= 0 && NNZ - index1(ii,kk,i) >= 0 && -i + N_R - 1 >= 0 && -k + N_C - 1 >= 0 && -j + index1(ii,kk,i) - 1 >= 0};
#codegen(Itile) given {[ii,kk,i,k,j]: N_R > 0 && N_C > 0 && index(ii,kk,i) >= 0 && index1(ii,kk,i) <= NNZ && index(ii,kk,i)+1 <= index1(ii,kk,i)};
#symbolic N_C,N_R,NNZ,NB,index(3),index1(3),col(5),b_row(1),b_row1(1),b_col(2);
#Iexec := { [ii,jj,i,k]: exists(ri,ck: 0 <= ri < 8 && i = ii*8+ri && 0 <= ck < 8 && k = b_col(ii,jj)*8+ck) && b_row(ii) <= jj < b_row1(ii) && i >= 0 && k >= 0 && -i + N_R - 1 >= 0 && -k + N_C - 1 >= 0};
#codegen(Iexec) given {[ii,jj,i,k]: b_col(ii,jj) >= 0 && 8*b_col(ii,jj) < N_C};
#symbolic N_C,N_R,NNZ,NB,index(2),index1(2),col(3);
#Icomp := { [ii,i,j,k]: exists(ri,ck: 0 <= ri < 8 && i = ii*8+ri && 0 <= ck < 8 && k = col(ii,i,j)) && i >= 0 && k >= 0 && index(ii,i) >= 0 && j - index(ii,i) >= 0 && NNZ - index1(ii,i) >= 0 && -i + N_R - 1 >= 0 && -k + N_C - 1 >= 0 && -j + index1(ii,i) - 1 >= 0};
#codegen(Icomp) given {[ii,i,j,k]: NNZ > 0 && N_C > 0 && index(ii,i) >= 0 && index1(ii,i) <= NNZ && col(ii,i,j) >= 0 && col(ii,i,j) < N_C};

# Icsr
# symbolic NNZ,N_R,index(1),index1(1);
# spmv := {[i,j]:0<=i<N_R&&index(i)<=j<index1(i)};
# codegen(spmv) given {[i,j]: index(i)>=0&&NNZ>0&&NNZ>=index1(i)};

omcode = """
#symbolic NNZ,N_R,index(1),index1(1);
#spmv := {[i,j]:0<=i<N_R&&index(i)<=j<index1(i)};
#codegen(spmv) given {[i,j]: index(i)>=0&&NNZ>0&&NNZ>=index1(i)};
symbolic N_C,N_R,b_col(2),b_row(1),b_row1(1);
Iexec := {[ii,jj,i,k]:exists(ri,ck:0<=ri<8&&i=ii*8+ri&&0<=ck<8&&k=b_col(ii,jj)*8+ck)&&b_row(ii)<=jj<b_row1(ii)&&i>=0&&k>=0&&-i+N_R-1>=0&&-k+N_C-1>=0};
codegen(Iexec) given {[ii,jj,i,k]: b_col(ii,jj)>=0&&N_C>0&&N_C>8*b_col(ii,jj)};
"""
run('/usr/local/bin/omegacalc', omcode, verbose=True)


Output:

>>> #symbolic NNZ,N_R,index(1),index1(1);
>>> #spmv := {[i,j]:0<=i<N_R&&index(i)<=j<index1(i)};
>>> #codegen(spmv) given {[i,j]: index(i)>=0&&NNZ>0&&NNZ>=index1(i)};
>>> symbolic N_C,N_R,b_col(2),b_row(1),b_row1(1);
>>> Iexec := {[ii,jj,i,k]:exists(ri,ck:0<=ri<8&&i=ii*8+ri&&0<=ck<8&&k=b_col(ii,jj)*8+ck)&&b_row(ii)<=jj<b_row1(ii)&&i>=0&&k>=0&&-i+N_R-1>=0&&-k+N_C-1>=0};
>>> codegen(Iexec) given {[ii,jj,i,k]: b_col(ii,jj)>=0&&N_C>0&&N_C>8*b_col(ii,jj)};
for(t1 = 0; t1 <= intFloor(N_R-1,8); t1++) {
  for(t2 = b_row(t1); t2 <= b_row1(t1)-1; t2++) {
    for(t3 = 8*t1; t3 <= min(N_R-1,8*t1+7); t3++) {
      for(t4 = 8*b_col(t1,t2); t4 <= min(8*b_col(t1,t2)+7,N_C-1); t4++) {
        s0(t1,t2,t3,t4);
      }
    }
  }
}

>>> 



('>>> #symbolic NNZ,N_R,index(1),index1(1);\n>>> #spmv := {[i,j]:0<=i<N_R&&index(i)<=j<index1(i)};\n>>> #codegen(spmv) given {[i,j]: index(i)>=0&&NNZ>0&&NNZ>=index1(i)};\n>>> symbolic N_C,N_R,b_col(2),b_row(1),b_row1(1);\n>>> Iexec := {[ii,jj,i,k]:exists(ri,ck:0<=ri<8&&i=ii*8+ri&&0<=ck<8&&k=b_col(ii,jj)*8+ck)&&b_row(ii)<=jj<b_row1(ii)&&i>=0&&k>=0&&-i+N_R-1>=0&&-k+N_C-1>=0};\n>>> codegen(Iexec) given {[ii,jj,i,k]: b_col(ii,jj)>=0&&N_C>0&&N_C>8*b_col(ii,jj)};\nfor(t1 = 0; t1 <= intFloor(N_R-1,8); t1++) {\n  for(t2 = b_row(t1); t2 <= b_row1(t1)-1; t2++) {\n    for(t3 = 8*t1; t3 <= min(N_R-1,8*t1+7); t3++) {\n      for(t4 = 8*b_col(t1,t2); t4 <= min(8*b_col(t1,t2)+7,N_C-1); t4++) {\n        s0(t1,t2,t3,t4);\n      }\n    }\n  }\n}\n\n>>> \n',
 '')

In [2]:
import re
from codegen.iegenlib import IEGenLib
from tools.system import run

omcode = """
symbolic L,NB,NNZ,bind1(1),bind2(1),bind3(1),bptr(1),bptr1(1),crd1(5),crd2(5),crd3(5),eind1(5),eind2(5),eind3(5);
Ihi := {[b,bi,bj,bk,m,ei,ej,ek,i,j,k,l]:bi-bind1(b)=0&&bj-bind2(b)=0&&bk-bind3(b)=0&&ei-eind1(b,bi,bj,bk,m)=0&&ej-eind2(b,bi,bj,bk,m)=0&&ek-eind3(b,bi,bj,bk,m)=0&&8bi+ei-i=0&&8bj+ej-j=0&&8bk+ek-k=0&&b>=0&&m>=0&&l>=0&&bptr(b)>=0&&m-bptr(b)>=0&&L-1>=0&&NNZ-bptr1(b)>=0&&NNZ-2>=0&&-b+NB-1>=0&&-m+NNZ-1>=0&&-m+bptr1(b)-1>=0&&-l+L-1>=0};
codegen(Ihi) given {[b,bi,bj,bk,m,ei,ej,ek,i,j,k,l]: bptr(b)>=0&&L-1>=0&&NNZ-bptr1(b)>=0&&NNZ-2>=0&&i-crd1(b,bi,bj,bk,m)=0&&j-crd2(b,bi,bj,bk,m)=0&&k-crd3(b,bi,bj,bk,m)=0};
"""
run('/usr/local/bin/omegacalc', omcode, verbose=True)


Output:

>>> symbolic L,NB,NNZ,bind1(1),bind2(1),bind3(1),bptr(1),bptr1(1),crd1(5),crd2(5),crd3(5),eind1(5),eind2(5),eind3(5);
>>> Ihi := {[b,bi,bj,bk,m,ei,ej,ek,i,j,k,l]:bi-bind1(b)=0&&bj-bind2(b)=0&&bk-bind3(b)=0&&ei-eind1(b,bi,bj,bk,m)=0&&ej-eind2(b,bi,bj,bk,m)=0&&ek-eind3(b,bi,bj,bk,m)=0&&8bi+ei-i=0&&8bj+ej-j=0&&8bk+ek-k=0&&b>=0&&m>=0&&l>=0&&bptr(b)>=0&&m-bptr(b)>=0&&L-1>=0&&NNZ-bptr1(b)>=0&&NNZ-2>=0&&-b+NB-1>=0&&-m+NNZ-1>=0&&-m+bptr1(b)-1>=0&&-l+L-1>=0};
>>> codegen(Ihi) given {[b,bi,bj,bk,m,ei,ej,ek,i,j,k,l]: bptr(b)>=0&&L-1>=0&&NNZ-bptr1(b)>=0&&NNZ-2>=0&&i-crd1(b,bi,bj,bk,m)=0&&j-crd2(b,bi,bj,bk,m)=0&&k-crd3(b,bi,bj,bk,m)=0};
for(t1 = 0; t1 <= NB-1; t1++) {
  t2=bind1(t1);
  t3=bind2(t1);
  t4=bind3(t1);
  for(t5 = bptr(t1); t5 <= bptr1(t1)-1; t5++) {
    if (crd3(t1,t2,t3,t4,t5) == 8*t4+eind3(t1,t2,t3,t4,t5) && crd1(t1,t2,t3,t4,t5) == 8*bind1(t1)+eind1(t1,t2,t3,t4,t5) && crd2(t1,t2,t3,t4,t5) == 8*bind2(t1)+eind2(t1,t2,t3,t4,t5)) {
      t6=crd1(t1,t2,t3,t4,t5)-8*bind1(t1);
    

('>>> symbolic L,NB,NNZ,bind1(1),bind2(1),bind3(1),bptr(1),bptr1(1),crd1(5),crd2(5),crd3(5),eind1(5),eind2(5),eind3(5);\n>>> Ihi := {[b,bi,bj,bk,m,ei,ej,ek,i,j,k,l]:bi-bind1(b)=0&&bj-bind2(b)=0&&bk-bind3(b)=0&&ei-eind1(b,bi,bj,bk,m)=0&&ej-eind2(b,bi,bj,bk,m)=0&&ek-eind3(b,bi,bj,bk,m)=0&&8bi+ei-i=0&&8bj+ej-j=0&&8bk+ek-k=0&&b>=0&&m>=0&&l>=0&&bptr(b)>=0&&m-bptr(b)>=0&&L-1>=0&&NNZ-bptr1(b)>=0&&NNZ-2>=0&&-b+NB-1>=0&&-m+NNZ-1>=0&&-m+bptr1(b)-1>=0&&-l+L-1>=0};\n>>> codegen(Ihi) given {[b,bi,bj,bk,m,ei,ej,ek,i,j,k,l]: bptr(b)>=0&&L-1>=0&&NNZ-bptr1(b)>=0&&NNZ-2>=0&&i-crd1(b,bi,bj,bk,m)=0&&j-crd2(b,bi,bj,bk,m)=0&&k-crd3(b,bi,bj,bk,m)=0};\nfor(t1 = 0; t1 <= NB-1; t1++) {\n  t2=bind1(t1);\n  t3=bind2(t1);\n  t4=bind3(t1);\n  for(t5 = bptr(t1); t5 <= bptr1(t1)-1; t5++) {\n    if (crd3(t1,t2,t3,t4,t5) == 8*t4+eind3(t1,t2,t3,t4,t5) && crd1(t1,t2,t3,t4,t5) == 8*bind1(t1)+eind1(t1,t2,t3,t4,t5) && crd2(t1,t2,t3,t4,t5) == 8*bind2(t1)+eind2(t1,t2,t3,t4,t5)) {\n      t6=crd1(t1,t2,t3,t4,t5)-8*bind1(t1);\n 

In [33]:
import re
from codegen.iegenlib import IEGenLib
from tools.system import run

icsr = 'Icsr := {[i,j]: 0 <= i < N_R && index(i) <= j < index(i+1)}'
idense = 'Idense := {[i,k,j]: k - col(j) = 0 && i >= 0 && k >= 0 && j - index(i) >= 0 && -i + N_R - 1 >= 0 && -k + N_C - 1 >= 0 && -j + index(i + 1) - 1 >= 0 }'
#ttile = 'Ttile := {[i,k,j] -> [ii,kk,i,k,j]: 0 <= ii < floor(N_R/8) && 0 <= kk < floor(N_C/8) && 8 * ii <= i < min(N_R - 1, 8 * ii + 7) && 8 * kk <= k < min(N_C - 1, 8 * kk + 7)}'
ttile = 'Ttile := {[i,k,j] -> [ii,kk,ri,ck,j]: 0 <= ii < N_R && 0 <= ri < R && i = ii*8+ri && 0 <= kk < N_C && 0 <= ck < C && k = kk*8+ck}'
# Perform the tile transformation with IEGenLib/Omega:
iegen = IEGenLib()
iegen.add(idense)
iegen.add(ttile)
(resid, result) = iegen.apply('Ttile', 'Idense', 'Itile')
assign = '%s := %s;' % (resid, result)
print("(IEGenLib) " + assign)
#omcode = iegen.to_omega(resid)

# CHiLL Example:
# sym = n;
# s1:={[In_1,In_2,In_3,In_4] : exists ( t2,t4,t6,t8,t2',t4',t6',t8' : ( exists ( alpha,beta : t2 = 16beta && t2' = 16alpha && t4' = t4 && t6' = 1+t6 && t8' = 1+t8 && t2' = t2+In_1 && t4' = t4+In_2 && t6' = t6+In_3 && t8' = t8+In_4 && 0, t8-15 <= t2 <= t8 <= t2'+14 && 0 <= t2' <= t8+1, n-1 && 0 <= t6 <= n-2 && 0 <= t4 < n && t2 < n) )) };
# s2:={[t8]: 0 < t8 < 8};

omcode = """
symbolic N_C,N_R,R,C,index(1),col(3),index1(1);
Idense := { [i, k, j] : k - col(i,k,j) = 0 && i >= 0 && k >= 0 && j - index(i) >= 0 && -i + N_R - 1 >= 0 && -k + N_C - 1 >= 0 && -j + index1(i) - 1 >= 0 };
#Ttile := {[i,k,j] -> [ii,kk,i,k,j]: exists(ri,ck: 0 <= ri < 8 && i = ii*8+ri && 0 <= ck < 8 && k = kk*8+ck)};
Ttile := {[i,k,j] -> [ii,kk,ri,ck,j]: 0 <= ii < N_R/8 && 0 <= ri < R && i = ii*8+ri && 0 <= kk < N_C/8 && 0 <= ck < C && k = kk*8+ck};
Itile := Ttile(Idense);
codegen(Itile);
"""
run('/usr/local/bin/omegacalc', omcode, verbose=True)


(IEGenLib) Itile := { [ii, kk, ri, ck, j] : 8 kk + ck - col(j) = 0 && ii >= 0 && kk >= 0 && ri >= 0 && ck >= 0 && 8 ii + ri >= 0 && 8 kk + ck >= 0 && j - index(8 ii + ri) >= 0 && -ii + N_R - 1 >= 0 && -kk + N_C - 1 >= 0 && -ri + R - 1 >= 0 && -ck + C - 1 >= 0 && -j + index(8 ii + ri + 1) - 1 >= 0 && -8 ii - ri + N_R - 1 >= 0 && -8 kk - ck + N_C - 1 >= 0 };
Output:

>>> symbolic N_C,N_R,R,C,index(1),col(3),index1(1);
>>> Idense := { [i, k, j] : k - col(i,k,j) = 0 && i >= 0 && k >= 0 && j - index(i) >= 0 && -i + N_R - 1 >= 0 && -k + N_C - 1 >= 0 && -j + index1(i) - 1 >= 0 };
>>> #Ttile := {[i,k,j] -> [ii,kk,i,k,j]: 0 <= ii < floor(N_R/8) && 0 <= kk < floor(N_C/8) && 8 * ii <= i < min(N_R - 1, 8 * ii + 7) && 8 * kk <= k < min(N_C - 1, 8 * kk + 7)};
>>> #Ttile := {[i,k,j] -> [ii,kk,i,k,j]: exists(ri,ck: 0 <= ri < 8 && i = ii*8+ri && 0 <= ck < 8 && k = kk*8+ck)};
>>> Ttile := {[i,k,j] -> [ii,kk,ri,ck,j]: 0 <= ii < N_R/8 && 0 <= ri < R && i = ii*8+ri && 0 <= kk < N_C/8 && 0 <= ck < C && k = 

('>>> symbolic N_C,N_R,R,C,index(1),col(3),index1(1);\n>>> Idense := { [i, k, j] : k - col(i,k,j) = 0 && i >= 0 && k >= 0 && j - index(i) >= 0 && -i + N_R - 1 >= 0 && -k + N_C - 1 >= 0 && -j + index1(i) - 1 >= 0 };\n>>> #Ttile := {[i,k,j] -> [ii,kk,i,k,j]: 0 <= ii < floor(N_R/8) && 0 <= kk < floor(N_C/8) && 8 * ii <= i < min(N_R - 1, 8 * ii + 7) && 8 * kk <= k < min(N_C - 1, 8 * kk + 7)};\n>>> #Ttile := {[i,k,j] -> [ii,kk,i,k,j]: exists(ri,ck: 0 <= ri < 8 && i = ii*8+ri && 0 <= ck < 8 && k = kk*8+ck)};\n>>> Ttile := {[i,k,j] -> [ii,kk,ri,ck,j]: 0 <= ii < N_R/8 && 0 <= ri < R && i = ii*8+ri && 0 <= kk < N_C/8 && 0 <= ck < C && k = kk*8+ck};\nsyntax error at line 5\n...skipping to statement end...\n>>> Itile := Ttile(Idense);\nrelation Ttile not declared at line 6\n...skipping to statement end...\n>>> codegen(Itile);\nrelation Itile not declared at line 7\n...skipping to statement end...\n>>> \n',
 '')

In [17]:
# spmv code
code = """
symbolic N,NNZ,col(1),index(1),ip1;
I_csr := {[i,j] : 0 <= i < N && 0 <= j < NNZ && index(i) <= j < index(ip1) && k = col(j)};
A := {[j] : 0 <= j < NNZ};
col := {[j] : 0 <= j < NNZ};
index := {[i] : 0 <= i < N};
x := {[k] : 0 <= k < N};
spmv := {[i,j] : 0 <= i < N && 0 <= j < NNZ && index(i) <= j < index(ip1) && k = col(j)};
codegen(spmv);
y := {[i] : 0 <= i < N};
"""

# dmxpy code
dmvcode = """
symbolic M,N;
I_dmv := {[i,k] : 0 <= i < M && 0 <= k < N};
codegen I_dmv;
"""

csrcode = """
# Define constants and variables
symbolic NNZ,N_R,N_C,R,C,k;

# Define uninterpreted functions...
symbolic col(1),index(1),ip1,index_i_,index_ip1_;

# Appears assignments can be handled like this...
NNZ := {[NNZ] -> [7] };

# I_csr iteration space
I_csr := {[i,j] : 0 <= i < N_R and index(i) <= j < index(ip1)};
I_csr;

# I_tile iteration space
# I_tile := {[ii,kk] : 0 <= ii < N_R/R and 0 <= kk < N_C/C };
# Seems division is not supported, so replace '/' with 'd'
#symbolic N_RdR,N_CdC;
#I_tile := {[ii,kk] : 0 <= ii < N_RdR and 0 <= kk < N_CdC };

# Codegen works...
codegen I_csr;

# Transformation function:
#T_csr_tile := {[i,j] -> [ii,kk] : exists(j : k=col(j)) and ii=floor(i/R) and kk=floor(k/C)};

"""

ex1code = """
symbolic N;
symbolic F(1);
S := {[i,j]  : 0<=i<N and 0<=j<F(i)};
codegen S;
"""

ex2code = """
symbolic i,ip1,N_R;
symbolic index(1);
I_csr := {[i,j] : 0 <= i < N_R and index(i) <= j < index(ip1)};
codegen I_csr;
"""

#print("Input:")
#print(code);

cmd = '/usr/local/bin/oc'# + ' --help'
import subprocess as sub
proc = sub.Popen(cmd, shell=True, stdin=sub.PIPE, stdout=sub.PIPE, stderr=sub.PIPE)
data = proc.communicate(input=code.encode('ascii'))
out = data[0].decode()
err = data[1].decode()

if len(out) > 0:
    print("Output:")
    print("=========================================================================================================\n")
    print(out)

if len(err) > 0:
    print("Error:")
    print("=========================================================================================================\n")
    print(err)


Output:

>>> symbolic N,NNZ,col(1),index(1),ip1;
>>> I_csr := {[i,j] : 0 <= i < N && 0 <= j < NNZ && index(i) <= j < index(ip1) && k = col(j)};
arguments to function must be prefix of input or output tuple at line 3
...skipping to statement end...
>>> A := {[j] : 0 <= j < NNZ};
>>> col := {[j] : 0 <= j < NNZ};
>>> index := {[i] : 0 <= i < N};
>>> x := {[k] : 0 <= k < N};
>>> spmv := {[i,j] : 0 <= i < N && 0 <= j < NNZ && index(i) <= j < index(ip1) && k = col(j)};
arguments to function must be prefix of input or output tuple at line 8
...skipping to statement end...
>>> codegen(spmv);
relation spmv not declared at line 9
...skipping to statement end...
>>> y := {[i] : 0 <= i < N};
>>> 

